In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
def ProPublica_API_access (query_string, ntee_code):
    """
    This function retrieves organizations from the ProPublica API matching a given query string, i.e. 'opera', 'symphony', etc.
    
    Args:
    query_string (str): A query string for the name of the organization.
    
    Returns:
    A Pandas dataframe 
    """
    
    url = 'https://projects.propublica.org/nonprofits/api/v2/search.json?q='+query_string
    organizations = []
    for x in range(11):
        params = dict(q=query_string, page=x)
        request = requests.get(url, params=params)
        json = request.json()
        json.pop('total_results')
        page_of_orgs = json.pop('organizations')
        organizations.append(page_of_orgs)
    global df
    df = pd.DataFrame()
    for x in range(len(organizations)):
        for y in range(len(organizations[x])):
            org = organizations[x][y]
            df = df.append(org,ignore_index=True)
    #This step will fix typos in the 'ntee_code' column#
    df['ntee_code'].fillna(ntee_code,inplace=True)
    df[df['ntee_code'].str.contains(ntee_code)]
    #This step sets the organization's EIN as the index#
    df['ein'] = df['ein'].astype('int')
    df = df.set_index('ein')
    return df.head()

In [3]:
ProPublica_API_access('opera','A6A')

,city,has_subseccd,have_extracts,have_filings,have_pdfs,name,ntee_code,raw_ntee_code,score,state,strein,sub_name,subseccd
ein,,,,,,,,,,,,,
813311094,ASTORIA,1.0,None,None,None,BARE OPERA,A6A,A6A,8022.336,NY,81-3311094,BARE OPERA,3.0
263016871,LEXINGTON,1.0,None,None,None,BLUEGRASS OPERA,A6A,A6A,8022.336,KY,26-3016871,BLUEGRASS OPERA,3.0
833139772,SUNLAND,1.0,None,None,None,MISSION OPERA,A6A,A6A,8022.336,CA,83-3139772,MISSION OPERA,3.0
462541609,BEND,1.0,None,None,None,OPERA BEND,A99,A99,8022.336,Oregon,46-2541609,OPERA BEND,3.0
840850527,ENGLEWOOD,1.0,None,None,None,OPERA COLORADO,A6A0,A6A0,8022.336,CO,84-0850527,OPERA COLORADO,3.0


In [4]:
eins = df.index.values.tolist()
eins

[813311094,
 263016871,
 833139772,
 462541609,
 840850527,
 464332937,
 453361602,
 222519789,
 800432320,
 824079263,
 237314812,
 850391704,
 821536854,
 824602315,
 910760426,
 752253647,
 364454720,
 463486730,
 814363604,
 260006536,
 953854105,
 566019660,
 432032363,
 231504706,
 133819493,
 522175213,
 20484003,
 462223000,
 930679959,
 475311222,
 471255941,
 237207572,
 454913985,
 203975320,
 10496860,
 541618192,
 952224201,
 943068290,
 726021455,
 200156714,
 824935291,
 952503791,
 954512214,
 463142361,
 50453691,
 510213334,
 571129329,
 411375724,
 680454224,
 263649993,
 756004746,
 522001426,
 471571156,
 721584393,
 222270100,
 270940187,
 920148120,
 237169261,
 832435880,
 954868027,
 262584694,
 341816013,
 811599946,
 770668548,
 463775780,
 223954352,
 475324014,
 942495701,
 990197758,
 263346265,
 261577184,
 461846033,
 833135996,
 260348842,
 821447688,
 270813566,
 382052726,
 272295236,
 621119830,
 886009267,
 942985556,
 203520577,
 300540629,
 133098

In [5]:
company_ein=[]
tax_period=[]
tot_assets=[]
tot_income=[]
tot_revenue=[]
pct_compnsatncurrofcr=[]
tax_year = []
totassetsend=[]
totliabend=[]
totfuncexpns=[]
skipped_requests=[]
for ein in eins:
    try:
        url2 = 'https://projects.propublica.org/nonprofits/api/v2/organizations/{}.json'.format(ein)
        request2 = requests.get(url2)
        json2 = request2.json()
        if len(json2['filings_with_data']) == 0:
            company_ein.append(ein)
            tax_period.append(json2['organization']['tax_period'])
            tot_assets.append(json2['organization']['asset_amount'])
            tot_income.append(json2['organization']['income_amount'])
            tot_revenue.append(json2['organization']['revenue_amount'])
            tax_year.append(np.nan)
            pct_compnsatncurrofcr.append(np.nan)
            totassetsend.append(np.nan)
            totliabend.append(np.nan)
            totfuncexpns.append(np.nan)
        else:
            for filing in range(len(json2['filings_with_data'])):
                company_ein.append(ein)
                tax_period.append(json2['organization']['tax_period'])
                tot_assets.append(json2['organization']['asset_amount'])
                tot_income.append(json2['organization']['income_amount'])
                tot_revenue.append(json2['organization']['revenue_amount'])
                tax_year.append(json2['filings_with_data'][filing]['tax_prd_yr'])
                pct_compnsatncurrofcr.append(json2['filings_with_data'][filing]['pct_compnsatncurrofcr'])
                totassetsend.append(json2['filings_with_data'][filing]['totassetsend'])
                totliabend.append(json2['filings_with_data'][filing]['totliabend'])
                totfuncexpns.append(json2['filings_with_data'][filing]['totfuncexpns'])
    except:
        skipped_requests.append(ein)
    finally:
        pass

In [6]:
df2 = pd.DataFrame(
    list(zip(company_ein, tax_period, tot_assets, tot_income, tot_revenue, pct_compnsatncurrofcr, tax_year, totassetsend, totliabend, totfuncexpns)),
    columns = ['EIN','tax_period', 'tot_assets', 'tot_income', 'tot_revenue', 'pct_compnsatncurrofcr', 'tax_year', 'totassetsend', 'totliabend', 'totfuncexpns'])
df2.head()

,EIN,tax_period,tot_assets,tot_income,tot_revenue,pct_compnsatncurrofcr,tax_year,totassetsend,totliabend,totfuncexpns
0,813311094,2019-12-01,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,263016871,2018-12-01,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,833139772,2019-12-01,1280.0,26521.0,26521.0,NaN,NaN,NaN,NaN,NaN
3,462541609,2018-12-01,12908.0,53745.0,50416.0,-0.001,2017.0,7598.0,6000.0,65239.0
4,462541609,2018-12-01,12908.0,53745.0,50416.0,-0.001,2015.0,50904.0,0.0,91185.0


In [7]:
df2.shape

(1808, 10)

In [8]:
df2.to_csv('Form 990s.csv')